In [ ]:
import os
import sqlite3
import subprocess
from github import Github

conn = sqlite3.connect('test_results.db')

conn.execute('''
CREATE TABLE IF NOT EXISTS results
(id INTEGER PRIMARY KEY AUTOINCREMENT, 
module TEXT NOT NULL, 
exercise TEXT NOT NULL, 
result TEXT NOT NULL);
''')

conn.commit()
conn.close
g = Github()  
def compile_and_test(module, repo_name, repo_owner):

    repo = g.get_user(repo_owner).get_repo(repo_name)
    #add logic to check not duplicated
    subprocess.check_call(f'git clone https://github.com/{repo_owner}/{repo_name}.git', shell=True)

    # Navigate into the repository
    os.chdir(repo_name)

    # For each exercise in the module
    for content in repo.get_contents(module):
        if content.type == "dir":
            exercise = content.name
            os.chdir(exercise)
            
            # Compile the C code
                        
            
            # Save the result in the database
            conn.execute('''
            INSERT INTO results (module, exercise, result) 
            VALUES (?, ?, ?);
            ''', (module, exercise, 'Pass' if result == 0 else 'Fail'))
            
            # Navigate back up to the module directory
            os.chdir('..')
            
    # Commit the changes to the database
    conn.commit()

    # Navigate back to the original directory
    os.chdir('..')

    # Remove the cloned repository
    subprocess.check_call(f'rm -rf {repo_name}', shell=True)

compile_and_test('module1', 'repo_name')  # replace 'module1' and 'repo_name' with your module and repo names


In [34]:
import os
import shutil
import subprocess
import git

def run_tests_on_repo(repo_url, test_path):
    # clone the repo
    repo_name = repo_url.split('/')[-1].split('.')[0]  # get repo name from url
    g = git.cmd.Git(os.getcwd())
    g.clone(repo_url)

    # Walk through the cloned repo directory
    for dirpath, _, filenames in os.walk(repo_name):
        for file in filenames:
            # Find the required source files
            if file.endswith(".c") and not file == "main.c":
                function_name = os.path.splitext(file)[0]  # get function name from file
                src_file_path = os.path.join(dirpath, file)
                
                # Construct the corresponding path in the test directory
                relative_dirpath = os.path.relpath(dirpath, repo_name)
                test_main_file_path = os.path.join(test_path, relative_dirpath, "main.c").replace("\\", "/")
                
                print(f"Expected test file location: {test_main_file_path}")  # For debugging
                
                # Copy main.c (test file) to source directory
                try:
                    shutil.copy(test_main_file_path, dirpath)
                except FileNotFoundError:
                    print(f"Test file for {function_name} not found.")
                    continue
                
                # Compile and run tests
                try:
                    subprocess.run(['gcc', '-o', function_name, src_file_path, os.path.join(dirpath, 'main.c')], check=True)
                    subprocess.run(['./' + function_name], check=True)
                    print(f"Test for {function_name} passed successfully.")
                except subprocess.CalledProcessError:
                    print(f"Test for {function_name} failed.")


In [39]:
repo = "https://github.com/Noeljarillo/bevolved01"

test = "./tests/0x01/"
run_tests_on_repo(repo, test)


Expected test file location: ./tests/0x01/00/main.c


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado